# Preenchimento Híbrido
Mostra o comportamento da rede utilizando os dois tipos de preenchimento. Essa simulação registra os recursos utilizados:

    - Número de Qubits criados
    - Número de EPRs criados

In [1]:
# Importando as dependências
from quantumnet.components import Controller, Network
from quantumnet.utils import generate_random_request, register_consumption, clear_file
from quantumnet.objects import time

Informações para a simulação

In [2]:
# Caminho do arquivo de saída
path = "resultados/hibrido/qubits_eprs_sync.csv"
# Limpa o arquivo de saída
clear_file(path)
# Sobre a rede
rede = Network()
rede.n_initial_eprs = 10
rede.n_initial_qubits = 10
row, col = 3, 4
rede.set_ready_topology("Grade", row, col)
# Sobre o controlador
controlador = Controller(rede)
controlador.default_ttl = 100
# Sobre as requisições
fmin_range = (0.5, 1)
neprs_range = (2, 4)
num_requests = 100
requests = list(generate_random_request(len(rede.hosts)-1, fmin_range, neprs_range) for i in range(num_requests))

Preenchimento proativo

In [3]:
hosts = rede.hosts
for alice in hosts:
    for bob in hosts:
        controlador.add_match_route_rule_in_host_proactive(alice, bob, (0.5, 0.6), 5)
        controlador.add_match_route_rule_in_host_proactive(alice, bob, (0.8, 0.9), 5)

Funções responsáveis pela passagem de tempo

In [4]:
def update_rules(host, time_for_update):
    """
    Atualiza as regras de um host
    Args:
        host (Host): Host que terá as regras atualizadas
        time_for_update (int): Tempo para atualização das regras
    """
    # Percorre a tabela de fluxo do host
    for match in host.flow_table.copy():
        # Recebe o objeto da regra, somente
        rule = host.flow_table[match][1]
        # Incrementa a idade da regra
        for t in range(time_for_update):
            rule.increment_age()
        # Verifica se a regra expirou
        if rule.ttl <= rule.age:
            if rule.hit_count == 0:
                host.flow_table.pop(match)
                print(f"[Time {time.get_current_time()}] Regra {rule} expirada em {host}")
            else:
                print(f"Regra reutilizada {rule} em {host}")
                rule.hit_count = 0
                rule.age = rule.ttl/2

def update_time(time_for_update):
    """
    Atualiza o tempo e as regras

    Args:
        time_for_update (int): Tempo para incrementar o time-slot.
    """
    # Atualiza o tempo
    for t in range(time_for_update):
        time.increment()
        
        # Registra o consumo de recursos
        register_consumption(time.get_current_time(), rede.registry_of_resources, path)
        
        # Atualiza os recursos de 10 em 10
        if time.get_current_time() % 10 == 0:
            rede.refresh_resources(num_qubits=10, num_eprs=10)
            print(f"[Time {time.get_current_time()}] Recursos atualizados")
    # Atualiza as regras
    for host in rede.hosts:
        update_rules(rede.get_host(host), time_for_update)

Preenchimento reativo, baseado nas requests.

    - Demora 1 time-slot se a regra existe.
    - Demora 3 time-slots se uma nova regra precisa ser adicionada.

In [5]:
for request in requests:
        print(f"[Time {time.get_current_time()}] Processando requisição {request}...")
        alice = rede.get_host(request.alice)
        rule = alice.find_rule_by_request(request)

        if rule == False:  # Caso não exista um match na tabela
            request.starttime = time.get_current_time()
            update_time(3)
            controlador.add_match_route_rule_in_host_reactive(request)
            rule = alice.find_rule_by_request(request)
            controlador.run_rule(rule[1])
            request.endtime = time.get_current_time() 
            
        else:  # Caso já exista a regra
            request.starttime = time.get_current_time()
            update_time(1)
            controlador.run_rule(rule[1])
            request.endtime = time.get_current_time()
            # Registra no CSV como já registrado
        
        # Exibir informações da requisição
        print(f"Request {request}: Start Time = {request.starttime}, End Time = {request.endtime}")

[Time 0] Processando requisição R-D59D30...
Time-slot 1 registrado: Qubits = 120, EPRs = 170
[BasicRuleProactive] Passo: 1
[BasicRuleProactive] Ação: CreateEPR(1-0)
Request R-D59D30: Start Time = 0, End Time = 1
[Time 1] Processando requisição R-D59DF0...
Time-slot 2 registrado: Qubits = 120, EPRs = 171
[BasicRuleProactive] Passo: 1
[BasicRuleProactive] Ação: CreateEPR(10-6)
[BasicRuleProactive] Ação: CreateEPR(6-5)
[BasicRuleProactive] Ação: CreateEPR(5-4)
[BasicRuleProactive] Passo: 2
[BasicRuleProactive] Ação: Swap(10-6-5)
Running Swap(10-6-5)
Request R-D59DF0: Start Time = 1, End Time = 2
[Time 2] Processando requisição R-D59E80...
Time-slot 3 registrado: Qubits = 120, EPRs = 174
[BasicRuleProactive] Passo: 1
[BasicRuleProactive] Ação: CreateEPR(9-5)
[BasicRuleProactive] Ação: CreateEPR(5-6)
[BasicRuleProactive] Ação: CreateEPR(6-7)
[BasicRuleProactive] Passo: 2
[BasicRuleProactive] Ação: Swap(9-5-6)
Running Swap(9-5-6)
Request R-D59E80: Start Time = 2, End Time = 3
[Time 3] Proces

In [6]:
memory = rede.get_host(1).memory
for q in memory:
    print("Tempo de criação do qubit", q._creation_time)
    print("Tempo de vida", q._life_time)
    print("Fidelidade atual do qubit", q.get_current_fidelity())
    print("Qubit relaxado?", q._relaxed)
    print()

Tempo de criação do qubit 0
Tempo de vida 238
Fidelidade atual do qubit 0.09255057751034329
Qubit relaxado? True

Tempo de criação do qubit 0
Tempo de vida 238
Fidelidade atual do qubit 0.09255057751034329
Qubit relaxado? True

Tempo de criação do qubit 0
Tempo de vida 238
Fidelidade atual do qubit 0.09255057751034329
Qubit relaxado? True

Tempo de criação do qubit 0
Tempo de vida 238
Fidelidade atual do qubit 0.09255057751034329
Qubit relaxado? True

Tempo de criação do qubit 0
Tempo de vida 238
Fidelidade atual do qubit 0.09255057751034329
Qubit relaxado? True

Tempo de criação do qubit 0
Tempo de vida 238
Fidelidade atual do qubit 0.09255057751034329
Qubit relaxado? True

Tempo de criação do qubit 0
Tempo de vida 238
Fidelidade atual do qubit 0.09255057751034329
Qubit relaxado? True

Tempo de criação do qubit 0
Tempo de vida 238
Fidelidade atual do qubit 0.09255057751034329
Qubit relaxado? True

Tempo de criação do qubit 0
Tempo de vida 238
Fidelidade atual do qubit 0.09255057751034

In [11]:
eprs = rede.edges[0, 1]['eprs']
for epr in eprs:
    print("Tempo de criação do EPR", epr._creation_time)
    print("Tempo de vida", epr._life_time)
    print("Fidelidade atual do EPR", epr.get_current_fidelity())
    print("EPR relaxado?", epr._relaxed)
    print()

Tempo de criação do EPR 0
Tempo de vida 238
Fidelidade atual do EPR 0.09255057751034329
EPR relaxado? True

Tempo de criação do EPR 0
Tempo de vida 238
Fidelidade atual do EPR 0.09255057751034329
EPR relaxado? True

Tempo de criação do EPR 0
Tempo de vida 238
Fidelidade atual do EPR 0.09255057751034329
EPR relaxado? True

Tempo de criação do EPR 0
Tempo de vida 238
Fidelidade atual do EPR 0.09255057751034329
EPR relaxado? True

Tempo de criação do EPR 1
Tempo de vida 237
Fidelidade atual do EPR 0.09162968387750485
EPR relaxado? True

Tempo de criação do EPR 10
Tempo de vida 228
Fidelidade atual do EPR 0.10228420671553748
EPR relaxado? True

Tempo de criação do EPR 10
Tempo de vida 228
Fidelidade atual do EPR 0.10228420671553748
EPR relaxado? True

Tempo de criação do EPR 10
Tempo de vida 228
Fidelidade atual do EPR 0.10228420671553748
EPR relaxado? True

Tempo de criação do EPR 10
Tempo de vida 228
Fidelidade atual do EPR 0.10228420671553748
EPR relaxado? True

Tempo de criação do EPR 